In [1]:


"""

利用swift，微调数据集，需要进行定义，可以更改每个任务的prompt，也可以将一个样例的部分任务拆解出来当做一个任务（类似VQA）

具体参考：https://swift.readthedocs.io/zh-cn/latest/Multi-Modal/qwen2-vl%E6%9C%80%E4%BD%B3%E5%AE%9E%E8%B7%B5.html
"""



'\n\n利用swift，微调数据集，需要进行定义，可以更改每个任务的prompt，也可以将一个样例的部分任务拆解出来当做一个任务（类似VQA）\n\n具体参考：https://swift.readthedocs.io/zh-cn/latest/Multi-Modal/qwen2-vl%E6%9C%80%E4%BD%B3%E5%AE%9E%E8%B7%B5.html\n'

In [1]:

"""
整理单个任务的数据：车牌识别
"""

import os
import json
from random import sample


chepai_data_dir = "/root/autodl-tmp/vlm/dataset/origin_dataset/"


chepai_dir_list = [os.path.join(chepai_data_dir, dir_name) for dir_name in os.listdir(chepai_data_dir) if "ccpd" in dir_name]


words_list = [
    "A", "B", "C", "D", "E",
    "F", "G", "H", "J", "K", 
    "L", "M", "N", "P", "Q", 
    "R", "S", "T", "U", "V", 
    "W", "X", "Y", "Z", "0", 
    "1", "2", "3", "4", "5", 
    "6", "7", "8", "9" ]

con_list = [
    "皖", "沪", "津", "渝", "冀",
    "晋", "蒙", "辽", "吉", "黑",
    "苏", "浙", "京", "闽", "赣",
    "鲁", "豫", "鄂", "湘", "粤",
    "桂", "琼", "川", "贵", "云",
    "西", "陕", "甘", "青", "宁",
    "新"]

num_show = 10


image2plate = []
image_path_list = []
plate_list = []

"""
生成单个任务：
参考样例格式（https://swift.readthedocs.io/zh-cn/latest/Multi-Modal/qwen2-vl%E6%9C%80%E4%BD%B3%E5%AE%9E%E8%B7%B5.html）：
{"query": "<image>55555", "response": "66666", "images": ["image_path"]}
{"query": "eeeee<image>eeeee<image>eeeee", "response": "fffff", "history": [], "images": ["image_path1", "image_path2"]}
{"query": "EEEEE", "response": "FFFFF", "history": [["query1", "response2"], ["query2", "response2"]], "images": []}

"""
add_system_prompt = True

"""
在做单任务微调时，可以更改system提示词，当对多个任务进行微调时， 系统提示词可以不更改，这样就统一提示词
"""
plate_example_prompt_list = []

for dir_name in chepai_dir_list:
    for index, file_name in enumerate(os.listdir(dir_name)):
        
        if len(file_name.split('-')) != 7:
            continue
        _, _, bbox, points, label, _, _ = file_name.split('-')
        
        file_path = os.path.join(dir_name, file_name)
        
        label = label.split('_')
        con = con_list[int(label[0])]
        words = [words_list[int(_)] for _ in label[1:]]
        label_word = con+''.join(words)
        # print(file_path, label_word)
        image2plate.append([file_path, label_word])
        image_path_list.append(file_path)
        plate_list.append(label_word)
        
        example_with_prompt = {"query":"<image>你是一个车牌识别专家，现给定一张带有车牌的汽车照片，只打印车牌号，不要输出与车牌号无关的内容", "response": label_word, "images":[file_path]}
        plate_example_prompt_list.append(example_with_prompt)
        # if index > num_show:
        #     break


# 按照比例进行抽样
# 根据plate_list中元素的首字母进行抽样

plate2prompt = {}

for prompt_example in plate_example_prompt_list:
    plate = prompt_example["response"][0]
    if plate not in plate2prompt:
        plate2prompt[plate] = [prompt_example]
    else:
        plate2prompt[plate].append(prompt_example)



sample_num_for_plate = 1200
# 保存每个车牌对应的抽样样本
sample_plate_prompt_examples = []
for plate, plate_promts in plate2prompt.items():
    if len(plate_promts) > sample_num_for_plate:
        
        sample_plate_prompt_examples.extend(sample(plate_promts, sample_num_for_plate))
    else:
        sample_plate_prompt_examples.extend(plate_promts)




from sklearn.model_selection import train_test_split


train_plate, test_plate = train_test_split(sample_plate_prompt_examples, test_size=0.1)


def save_jsonl(data_list, save_file_name):

    with open(save_file_name, "w", encoding="utf-8") as fout:
        for line in data_list:
            fout.write(json.dumps(line, ensure_ascii=False) + "\n")
    

save_jsonl(train_plate, "./processed_dataset/plate_train.jsonl")
save_jsonl(test_plate, "./processed_dataset/plate_test.jsonl")

print("车牌保存完成")

车牌保存完成


In [3]:
train_plate[:2]

[{'query': '<image>你是一个车牌识别专家，现给定一张带有车牌的汽车照片，识别并输出车牌号，不要输出与车牌号无关的内容',
  'response': '皖A153J6',
  'images': ['/root/autodl-tmp/vlm/dataset/origin_dataset/ccpd_fn/0876-27_41-186&445_471&702-439&571_186&702_218&576_471&445-0_0_25_29_27_8_30-88-233.jpg']},
 {'query': '<image>你是一个车牌识别专家，现给定一张带有车牌的汽车照片，识别并输出车牌号，不要输出与车牌号无关的内容',
  'response': '皖AM829F',
  'images': ['/root/autodl-tmp/vlm/dataset/origin_dataset/ccpd_tilt/0471-30_40-400&317_601&513-585&406_400&513_416&424_601&317-0_0_11_32_26_33_5-122-153.jpg']}]

In [2]:
import random

prompts = [
    # 强调左前45度照片的提示
    "从这张左前45度角的照片中识别汽车的品牌和车系，输出格式：品牌-车系。",
    "请根据此车左前方45度视角的图片，识别品牌和车系，输出格式：品牌-车系。",
    "从这张45度左前角图片中，辨认汽车品牌和车系，格式为：品牌-车系。",
    "通过这张左前方45度的照片，判断汽车品牌和车系，输出格式：品牌-车系。",
    "根据左前45度角度的汽车照片，识别品牌与车系，输出为：品牌-车系。",
    "识别出此左前方45度角的汽车品牌和车系，要求输出格式：品牌-车系。",
    "从这张45度左前方角度图中分析汽车品牌和车系，格式要求：品牌-车系。",
    "根据这张左前方45度角度图，判断该车的品牌与车系，输出格式：品牌-车系。",
    "分析这张左前方45度角照片，识别汽车品牌和车系，输出为：品牌-车系。",
    "根据此45度左前角度的图像，确认汽车品牌和车系，格式：品牌-车系。",
    "从这张左前方45度角度的照片中，判断汽车品牌和车系，输出格式：品牌-车系。",
    "识别此车左前45度角照片中的品牌和车系，要求输出格式：品牌-车系。",
    "从左前45度视角的图片中，判断汽车的品牌和车系，输出格式：品牌-车系。",
    "根据左前45度角图片，分析汽车品牌和车系，输出格式为：品牌-车系。",
    "通过这张左前方45度角的汽车图片，识别品牌和车系，输出格式：品牌-车系。",
    "从此45度左前角照片中，确认该汽车的品牌与车系，输出格式：品牌-车系。",
    "分析这张汽车左前45度角度的图像，输出品牌和车系，格式：品牌-车系。",
    "根据左前45度角的照片，识别出汽车品牌和车系，输出格式：品牌-车系。",
    "请从这张左前45度角度的汽车图片中判断品牌和车系，输出格式为：品牌-车系。",
    "识别左前45度视角汽车的品牌和车系，格式要求：品牌-车系。",
    
    # 不强调左前45度照片的提示
    "请识别图中汽车的品牌和车系，输出格式：品牌-车系。",
    "根据这张汽车图片，判断品牌和车系，输出格式为：品牌-车系。",
    "识别此图中的汽车品牌和车系，要求输出：品牌-车系。",
    "分析图片中的汽车，提供品牌和车系，输出格式：品牌-车系。",
    "请确认图中车辆的品牌和车系，输出格式：品牌-车系。",
    "根据汽车照片，输出品牌和车系，格式为：品牌-车系。",
    "从图片中判断汽车的品牌和车系，输出格式：品牌-车系。",
    "请从这张汽车图片中识别品牌和车系，输出格式：品牌-车系。",
    "分析该汽车图像，识别品牌和车系，格式要求：品牌-车系。",
    "请确认此车的品牌和车系，输出结果为：品牌-车系。",
    "判断汽车照片中的品牌和车系，输出格式：品牌-车系。",
    "从这张图片中，识别该车的品牌和车系，输出格式：品牌-车系。",
    "请根据汽车图片，输出其品牌和车系，格式为：品牌-车系。",
    "分析图中的汽车，识别品牌和车系，输出格式：品牌-车系。",
    "请判断图片中的汽车品牌和车系，输出格式为：品牌-车系。",
    "从这张车的图片中，判断品牌和车系，输出格式：品牌-车系。",
    "识别汽车图片中的品牌与车系，输出为：品牌-车系。",
    "请根据图像，识别汽车的品牌和车系，输出格式：品牌-车系。",
    "通过照片判断该汽车的品牌与车系，输出格式：品牌-车系。",
    "请确认图中车辆品牌和车系，要求输出格式为：品牌-车系。"
]


"""
整理车型识别的数据
具体操作步骤参考上述：
"""
import os
import json
import random

carmodel_dir = "/root/autodl-tmp/vlm/dataset/origin_dataset/carmodel_ttpai/lf_datasets_output/"

num_list = []

def create_carmodel_dataset(data_dir_list, save_json_name):

    sample_carmodel_num = 60  # 每个品牌车系下选择60个照片

    all_prompt_examples = []
    
    
    for data_dir in data_dir_list:
        for carmodel_name_dir in os.listdir(data_dir):
            # carmodel_name_dir: 是由品牌车系组成的目录名称
            carmodel_split = carmodel_name_dir.split("-")
            if len(carmodel_split) != 2:
                continue
            carmodel_json = json.dumps({"品牌": carmodel_split[0], "车系": carmodel_split[1]}, ensure_ascii=False)
            
            carmodel_image_path = os.path.join(data_dir, carmodel_name_dir)
            num_list.append(len(os.listdir(carmodel_image_path)))
            
            image_path_for_carmodel = os.listdir(carmodel_image_path)

            if len(image_path_for_carmodel) > sample_carmodel_num:
                sample_image_path_for_carmodel = random.sample(image_path_for_carmodel, sample_carmodel_num)
            else:
                sample_image_path_for_carmodel = image_path_for_carmodel
            
            for carmodel_image_name in sample_image_path_for_carmodel:
                carmodel_image_file_path = os.path.join(carmodel_image_path, carmodel_image_name)
            
                # print(carmodel_name_dir, carmodel_image_path)
                # query_prompt = "<image>你是一个汽车品牌专家，精通熟悉各类汽车的品牌车系，包括油车和新能源车，现在给你一张汽车左前45度的图片，根据该图片识别该汽车的品牌和车系，仅输出品牌和车系名称，返回格式：品牌-车系，注意不要返回其它无关的信息"
                query_prompt = "<image>" + random.sample(prompts, 1)[0]
                one_input_example_prompt = {"query": query_prompt, "response": carmodel_name_dir, "images": [carmodel_image_file_path]}  # 使用原始的字符串拼接：品牌-车系
                
                # query_prompt = '<image>你是一个汽车品牌专家，精通熟悉各类汽车的品牌车系，包括油车和新能源车，现在给你一张汽车左前45度的图片，根据该图片识别该汽车的品牌和车系，您可以先识别汽车品牌然后再识别车系，或者一次性识别识别汽车的品牌和车系，仅输出品牌和车系名称，不要输出其它无关的内容，返回格式为：{“品牌": "", "车系": ""}'
                # one_input_example_prompt = {"query": query_prompt, "response": carmodel_json, "images": [carmodel_image_file_path]}   # 使用json字符串的格式进行生成
                all_prompt_examples.append(one_input_example_prompt)
    # print(all_prompt_examples[:5])
    for example in all_prompt_examples[:5]:
        print(example)
    with open(save_json_name, "w", encoding="utf-8") as fout:
        for example_prompt in all_prompt_examples:
            fout.write(json.dumps(example_prompt, ensure_ascii=False) + "\n")
    print("保存成功: ", save_json_name, "\n\n\n")


train_carmodel_dir = os.path.join(carmodel_dir, "train")
val_carmodel_dir = os.path.join(carmodel_dir, "val")
save_carmodel_train_json_name = os.path.join("/root/autodl-tmp/vlm/dataset/processed_dataset", "carmodel_train.jsonl")
create_carmodel_dataset([train_carmodel_dir, val_carmodel_dir], save_json_name=save_carmodel_train_json_name)

    
test_carmodel_dir = os.path.join(carmodel_dir, "test")

save_carmodel_test_json_name = os.path.join("/root/autodl-tmp/vlm/dataset/processed_dataset", "carmodel_test.jsonl")
create_carmodel_dataset([test_carmodel_dir], save_json_name=save_carmodel_test_json_name)




{'query': '<image>识别此图中的汽车品牌和车系，要求输出：品牌-车系。', 'response': 'AITO-问界M5', 'images': ['/root/autodl-tmp/vlm/dataset/origin_dataset/carmodel_ttpai/lf_datasets_output/train/AITO-问界M5/77131700_2023款 增程四驱智驾版.jpg']}
{'query': '<image>识别此车左前45度角照片中的品牌和车系，要求输出格式：品牌-车系。', 'response': 'AITO-问界M5', 'images': ['/root/autodl-tmp/vlm/dataset/origin_dataset/carmodel_ttpai/lf_datasets_output/train/AITO-问界M5/67796086_2022款 四驱性能版.jpg']}
{'query': '<image>识别此图中的汽车品牌和车系，要求输出：品牌-车系。', 'response': 'AITO-问界M5', 'images': ['/root/autodl-tmp/vlm/dataset/origin_dataset/carmodel_ttpai/lf_datasets_output/train/AITO-问界M5/78575303_2022款 增程后驱标准版.jpg']}
{'query': '<image>根据汽车照片，输出品牌和车系，格式为：品牌-车系。', 'response': 'AITO-问界M5', 'images': ['/root/autodl-tmp/vlm/dataset/origin_dataset/carmodel_ttpai/lf_datasets_output/train/AITO-问界M5/77278482_2022款 增程四驱性能版.jpg']}
{'query': '<image>通过这张左前方45度角的汽车图片，识别品牌和车系，输出格式：品牌-车系。', 'response': 'AITO-问界M5', 'images': ['/root/autodl-tmp/vlm/dataset/origin_dataset/carmodel_ttpai/lf_datasets_o

In [17]:
print(len(num_list))
# print(sorted(num_list))

s = 0

for i in num_list:
    s += min(i, 60)
print(s)


    

9272
176717


In [2]:
"""
查看有多少的品牌和车系
"""

import jsonlines

num_show = 20
i = 0

carmode2num = {}

num_train_example = 0
for example in jsonlines.open("processed_dataset/carmodel_test.jsonl", mode="r"):
    num_train_example += 1
    carmodel_name = example["response"]
    if carmodel_name not in carmode2num:
        carmode2num[carmodel_name] = 1
    else:
        carmode2num[carmodel_name] += 1

print("数据量：", num_train_example)
print("品牌车系的数量：", len(carmode2num))


print(sorted(carmode2num.items(), key=lambda x: x[1]))








数据量： 79099
品牌车系的数量： 2318
[('丰田-雷凌双擎E+', 12), ('Jeep-角斗士', 12), ('LEVC-LEVC TX', 12), ('五菱汽车-五菱Air ev晴空', 12), ('MG-MG领航', 12), ('五菱汽车-五菱佳辰', 12), ('五菱汽车-五菱征程经典', 12), ('五菱汽车-五菱征途', 12), ('五菱汽车-五菱星驰', 12), ('五菱汽车-五菱缤果', 12), ('五菱汽车-五菱荣光小卡翼开启', 12), ('ARCFOX极狐-极狐考拉', 12), ('BAW北汽制造-212', 12), ('北汽昌河-北斗星X5E', 12), ('北汽昌河-北汽EV5', 12), ('北汽昌河-北汽昌河M70', 12), ('北汽昌河-北汽昌河Q7', 12), ('北汽昌河-福瑞达M50', 12), ('北汽瑞翔-博腾V2 EV', 12), ('北汽绅宝-绅宝CC', 12), ('北汽绅宝-绅宝D20', 12), ('DS-DS 4', 12), ('北汽绅宝-绅宝X55', 12), ('北汽绅宝-绅宝X65', 12), ('北汽绅宝-绅宝智道', 12), ('华泰新能源-华泰EV160R', 12), ('华骐-华骐300E', 12), ('双龙-雷斯特W', 12), ('吉利几何-EX3 功夫牛', 12), ('Jeep-指挥官PHEV', 12), ('吉利几何-吉利几何M6', 12), ('奔腾-奔腾B70S', 12), ('Jeep-牧马人新能源', 12), ('奔腾-奔腾X40新能源', 12), ('MG-名爵EZS纯电动', 12), ('MG-名爵HS新能源', 12), ('WEY-WEY P8', 12), ('R汽车-MARVEL R', 12), ('一汽-森雅R9', 12), ('一汽-骏派A50', 12), ('一汽-骏派A70', 12), ('一汽-骏派CX65', 12), ('一汽-骏派D80', 12), ('万象汽车-万象T01', 12), ('三菱-LANCER', 12), ('三菱-伊柯丽斯', 12), ('三菱-帕杰罗速跑', 12), ('三菱-风迪思', 12), ('上汽大通MAXUS-上汽大通D